# Final Project - Super Blocks

In [1]:
# Import
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import floor
import patsy
import statsmodels.api as sm
from scipy.stats import ttest_ind

## Introduction and Background

### Many cities and neighbourhoods in the United States, such as Los Angeles have been designed for extensive car use. While these models might have worked in the past, this dynamic is beginning to change as the result of overpopulation, rideshares and automation among other factors. Urban neighborhoods are no longer just places of living for large amounts of people, but rather an intertwined mesh of commercial activity, transport and residences (3). This does not fit the originally intended model - it is not efficient, causes pollution, and prevents children from being able to play in the streets.

### In Barcelona(2), city planners realized this problem and implemented a system where they blocked off certain urban areas to vehicle traffic, opening them up for commercial and community use. The results were astounding - with an increase shown in community engagement along with a decrease in pollution. Economic benefits have also been shown for the local businesses in these "superblocks" (4,5). As a result, we have decided to create a model that can figure out the best blocks to block out for such activity, geospatially and temporally.

### We will be using the number of restaurants, coffee shops, bars and transit stations to categorize the 'walkability' of different blocks, building a model analagous to walkscore (1). Furthermore, we will be using the Yelp API to temporally analyze foot traffic based on numbers and timing of reviews and checkins.

##### Referencess:
  ######  1)https://www.walkscore.com/professional/research.php
   ###### 2)https://www.theguardian.com/cities/2016/may/17/superblocks-rescue-barcelona-spain-plan-give-streets-back-residents
   ###### 3)https://www.wired.com/2017/04/brilliant-simplicity-new-yorks-new-times-square/ 
   ###### 4)http://krqe.com/2017/03/06/city-councilor-wants-wider-sidewalks-to-help-businesses-impacted-by-art/
   ###### 5)http://www.nyc.gov/html/dot/downloads/pdf/dot-economic-benefits-of-sustainable-streets.pdf

## Data Description

'FY2014 Ridership_Trolley_Sept2013Booking.csv' has the information about the number of people getting on and off the trolley for each ride for Sept2013.

'FY2014 Ridership_Trolley_Sept2013_Stops.csv' has the information about the location and other information about the trolley locations in San Diego.

## Data Cleaning / Pre-processing

### Active Businesses (Restaurants, cafe's etc) 

In [9]:
#Importing the original file for active business data in SD

fname = "business.csv"
business_df = pd.read_csv(fname)

In [10]:
#Importing the original file for active business data in SD

fname1 = "stops.csv"
stops_df = pd.read_csv(fname1)

In [11]:
#Importing the original file for active business data in SD

fname2 = "parking.csv"
parking_df = pd.read_csv(fname2)

In [12]:
#Removing unneccesary columns and renaming to layman terms, filtering all business into places that serve food/drinks

business_df = business_df[['doing_bus_as_name','zip','naics_description','lat','lon']]
business_df = business_df.loc[(business_df['naics_description'] == 'full-service restaurants') |
               (business_df['naics_description'] == 'cafeterias') | 
               (business_df['naics_description'] == 'food services & drinking places') |
               (business_df['naics_description'] == 'limited-service eating places') |
               (business_df['naics_description'] == 'limited-service restaurants')  |
               (business_df['naics_description'] == 'mobile food services') |
               (business_df['naics_description'] == 'drinking places (alcoholic beverages)') |
               (business_df['naics_description'] == 'snack & nonalcoholic beverage bars')]
business_df.rename(columns = {'doing_bus_as_name':'Business title','naics_description':'Type of Place'}, inplace=True)
business_df = business_df.reset_index(drop=True)

In [13]:
#Adding the 5 columns for the different time brackets

business_df.insert(2,'AM early','Null')
business_df.insert(3,'AM peak','Null')
business_df.insert(4,'Mid-day','Null')
business_df.insert(5,'PM peak','Null')
business_df.insert(6,'PM late','Null')

In [14]:
#generalising the different categories into 3 categories: Only Food, Food & Drinks, Only Drinks

business_df.loc[(business_df['Type of Place'] == 'mobile food services') |
                (business_df['Type of Place'] == 'cafeterias') |
                (business_df['Type of Place'] == 'snack & nonalcoholic beverage bars') |
                (business_df['Type of Place'] == 'limited-service eating places') |
                (business_df['Type of Place'] == 'limited-service restaurants') 
                , 'Type of Place'] = 'Only Food'

business_df.loc[(business_df['Type of Place'] == 'full-service restaurants') 
                , 'Type of Place'] = 'Food & Drinks'

business_df.loc[(business_df['Type of Place'] == 'drinking places (alcoholic beverages)') |
                (business_df['Type of Place'] == 'food services & drinking places') 
                , 'Type of Place'] = 'Only Drinks'

In [15]:
#Assigning values to different time brackets by assuming foot traffic according to the type of place 

business_df.loc[(business_df['Type of Place'] == 'Only Food'), 
                ('AM early','AM peak','Mid-day','PM peak','PM late')] = ('3','25','20','18','5')
business_df.loc[(business_df['Type of Place'] == 'Food & Drinks'),
                ('AM early','AM peak','Mid-day','PM peak','PM late')] = ('7','28','25','35','25')
business_df.loc[(business_df['Type of Place'] == 'Only Drinks'),
                ('AM early','AM peak','Mid-day','PM peak','PM late')] = ('27','6','3','27','30') 

In [16]:
#Assigning a total score to all three categories (summ of scores of all time brackets)

business_df.loc[(business_df['Type of Place'] == 'Only Food'), 
                'Total Score'] = '71'
business_df.loc[(business_df['Type of Place'] == 'Food & Drinks'),
                'Total Score'] = '120'
business_df.loc[(business_df['Type of Place'] == 'Only Drinks'),
                'Total Score'] = '93'

In [17]:
business_df

,Business title,zip,AM early,AM peak,Mid-day,PM peak,PM late,Type of Place,lat,lon,Total Score
0,c r e a m,92115-1939,3,25,20,18,5,Only Food,32.767243,-117.096294,71
1,snow cones y raspados,92113-2911,3,25,20,18,5,Only Food,32.697974,-117.096250,71
2,jalapeno taco shop,92104-2047,7,28,25,35,25,Food & Drinks,32.748683,-117.126968,120
3,tacos el campechano inc,91950-1121,3,25,20,18,5,Only Food,32.697974,-117.096250,71
4,up2you cafe llc,92111-5000,3,25,20,18,5,Only Food,32.718370,-117.157817,71
5,awash ethiopian restaurant,92104-1102,7,28,25,35,25,Food & Drinks,32.677079,-117.107167,120
6,gueros taco shop,92102-4019,3,25,20,18,5,Only Food,32.890137,-117.150877,71
7,the fire spot,92111-2315,7,28,25,35,25,Food & Drinks,32.773069,-117.156144,120
8,up2you cafe llc,92111-1545,27,6,3,27,30,Only Drinks,33.016118,-117.075608,93
9,cold beers & cheeseburgers,92101-6910,7,28,25,35,25,Food & Drinks,32.708791,-117.160357,120


In [ ]:
business_df.to_csv('/Users/Devyaanshu/Pr_007/restaurants_data.csv')

### Transit Stops Dataset
The following cells take the original dataset and clean it so that we get the cleaned data, NOT MEANT TO BE RUN since we already have the cleaned dataset

##### The time ranges for which we have split the transit ridership data are:
AM_Early = 12AM-6AM
AM_Late = 6AM-9AM
..
..


In [ ]:
#stop_counts.to_csv('Stop_Counts.csv')

In [2]:
#cleaned dataset for the transit spots
#stops_df

### Parking meters Dataset 

In [7]:
#cleaned dataset for the parking meters

parking_df

,Unnamed: 0,total_count,pole_id,longitude,latitude,midday,am_early,am_peak,pm_peak,pm_late
0,0,10.767183,1-1004,-117.163929,32.715904,5.096620,0.298451,2.066197,3.191127,0.114789
1,1,11.283732,1-1006,-117.163930,32.716037,5.578732,0.286972,1.905493,3.455141,0.057394
2,2,10.927887,1-1008,-117.163931,32.716169,5.050704,0.137746,2.261338,3.432183,0.045915
3,3,10.044014,1-1020,-117.161278,32.717890,3.776549,0.160704,1.882535,4.166831,0.057394
4,4,12.362746,1-1310,-117.163951,32.719024,7.105423,0.573944,3.420704,1.251197,0.011479
5,5,12.718592,1-1312,-117.163952,32.719161,7.713803,0.321408,3.524014,1.124930,0.034437
6,6,12.282394,1-1313,-117.163770,32.719298,7.426831,0.309930,3.546972,0.975704,0.022958
7,7,13.326972,1-1314,-117.163953,32.719453,7.644930,0.516549,3.799507,1.354507,0.011479
8,8,11.111549,1-1315,-117.163772,32.719571,6.784014,0.149225,3.122254,1.056056,0.000000
9,9,11.203380,1-1317,-117.163772,32.719707,6.956197,0.195141,3.351831,0.700211,0.000000


## Data Visualization

## Data Analysis and Results 

## Conclusions/Discussion